In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
# Predict total bill >> target variable

In [5]:
# EDA >> subjective
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [7]:
X = df.drop('total_bill', axis = 1)
y = df['total_bill']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

In [9]:
X_train.shape

(195, 6)

In [10]:
X_test.shape

(49, 6)

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [12]:
cat_cols = ['sex', 'smoker', 'day', 'time']
num_cols = ['tip', 'size']

In [13]:
num_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy='median')),
                  ('scaling', StandardScaler())])


In [14]:
cat_pipeline = Pipeline(steps= [('imputation', SimpleImputer(strategy = 'most_frequent')),
                                ('encoding', OneHotEncoder())])

In [16]:
preprocessor = ColumnTransformer([('num_pipeline', num_pipeline, num_cols),
                                  ('cat_pipeline', cat_pipeline, cat_cols)])

In [17]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaling',
                                                  StandardScaler())]),
                                 ['tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day', 'time'])])

In [18]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [19]:
X_train

array([[-1.47443803, -0.57766863,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       [-0.71612531,  1.47042924,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.19880579,  1.47042924,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.43283335, -0.57766863,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.29060464, -0.57766863,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.1034652 ,  0.44638031,  1.        , ...,  0.        ,
         1.        ,  0.        ]])

In [20]:
X_test

array([[-1.48209775, -1.60171757,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [ 0.04984713, -0.57766863,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         1.        ,  0.        ],
       [ 0.36389583,  0.44638031,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [-0.33313909, -0.57766863,  0.        ,  1.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ],
       [ 0.04984713, -0.57766863,  0.        ,  1.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ],
       [-1.14506988, -0.57766863,  0.        ,  1.        ,  0.        ,
         1.        ,  

In [22]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [23]:
models = {"support vector regressor" : SVR(),
          "DT Regressor" : DecisionTreeRegressor(),
          "Multiple Linear Regression" : LinearRegression(),
          "Random_forest_Regressor" : RandomForestRegressor()}

In [24]:
models

{'support vector regressor': SVR(),
 'DT Regressor': DecisionTreeRegressor(),
 'Multiple Linear Regression': LinearRegression(),
 'Random_forest_Regressor': RandomForestRegressor()}

In [25]:
from sklearn.metrics import r2_score
def model_train_eval(X_train, X_test, y_train, y_test, models):
  evaluation = {}
  for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    model_score = r2_score(y_test, y_pred)
    evaluation[list(models.keys())[i]] = model_score
  return evaluation

In [28]:
model_train_eval(X_train, X_test, y_train, y_test, models)

{'support vector regressor': 0.4069433055293954,
 'DT Regressor': 0.1801086298272726,
 'Multiple Linear Regression': 0.6719006761625261,
 'Random_forest_Regressor': 0.6413328844248993}

In [29]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(oob_score=True)
rfr

RandomForestRegressor(oob_score=True)

In [30]:
from sklearn.model_selection import RandomizedSearchCV
params = {'max_depth' : [1, 50, 100, 150, 200],
          'n_estimators' : [50, 100, 200]}

In [31]:
reg = RandomizedSearchCV(rfr, param_distributions=params, cv = 5, verbose = 2, scoring='r2')

In [32]:
reg

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   param_distributions={'max_depth': [1, 50, 100, 150, 200],
                                        'n_estimators': [50, 100, 200]},
                   scoring='r2', verbose=2)

In [33]:
reg.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ....................max_depth=100, n_estimators=100; total time=   0.4s
[CV] END ....................max_depth=100, n_estimators=100; total time=   0.3s
[CV] END ....................max_depth=100, n_estimators=100; total time=   0.2s
[CV] END ....................max_depth=100, n_estimators=100; total time=   0.2s
[CV] END ....................max_depth=100, n_estimators=100; total time=   0.2s
[CV] END .....................max_depth=50, n_estimators=200; total time=   0.4s
[CV] END .....................max_depth=50, n_estimators=200; total time=   0.5s
[CV] END .....................max_depth=50, n_estimators=200; total time=   0.4s
[CV] END .....................max_depth=50, n_estimators=200; total time=   0.5s
[CV] END .....................max_depth=50, n_estimators=200; total time=   0.9s
[CV] END ....................max_depth=100, n_estimators=200; total time=   0.8s
[CV] END ....................max_depth=100, n_es

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   param_distributions={'max_depth': [1, 50, 100, 150, 200],
                                        'n_estimators': [50, 100, 200]},
                   scoring='r2', verbose=2)

In [34]:
reg.best_params_

{'n_estimators': 200, 'max_depth': 50}

In [35]:
reg.best_score_

np.float64(0.4042063819428063)

In [36]:
reg.best_estimator_

RandomForestRegressor(max_depth=50, n_estimators=200, oob_score=True)

In [37]:
final_model = reg.best_estimator_

In [38]:
final_model.predict(X_test)

array([10.0345    , 17.76620833, 25.3145    , 13.70683714, 20.75487083,
       11.27955   , 15.14005   , 24.3272    , 15.77899149, 22.66391667,
       17.79796667, 12.22823792, 29.6336    , 20.61055   , 36.00825   ,
       14.22262143, 16.00595   , 15.2847    , 13.57644524, 12.11087917,
       34.68836429, 18.0213    , 19.86455   , 33.37065   , 12.26236214,
       12.38206786, 18.55865   , 22.6942    , 17.5301625 , 19.75135   ,
       12.78737894, 31.63295   , 13.80961643, 26.1889    , 34.1896    ,
       12.92549583, 27.69735   , 20.1486919 , 20.969     , 12.38206786,
       21.53328929, 19.0641    , 20.248     , 21.4545    , 20.4857919 ,
       20.405975  , 21.00715   , 30.74715   , 16.130675  ])